<a href="https://colab.research.google.com/github/junsukim-me/2014100881/blob/master/hspi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
####################################
### junsu.kim backtest proj hspi ###
####################################
### date : 2021-06-14
###

from bs4 import BeautifulSoup
import requests, lxml, time, reprlib
from datetime import datetime, timedelta

import re as regexz
import operator

import pandas_datareader.data as web
import pandas as pd
import numpy as np

In [6]:



#re = reprlib.Repr()
#re.maxstring = 20    # max characters displayed for strings

def midReturn(val, s, e):
    if s in val:
        val = val[val.find(s)+len(s):]
        if e in val: val = val[:val.find(e)]
    return val

taskdone = False
trial = 0

# 탐색 날짜 범위 (ex. days=1 : 1일 이내, 0:측정 시작순간 이후)
# 설정 날짜의 딱 자정으로 설정됩니다 (ex. 8.18 1:45AM -> 8.18 00:00AM)
#drange = int(input('탐색 날짜 범위 (ex. 1=1일 이내): '))
drange = 100

fontpath='font.otf'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36'
}

# 갤러리ID
#gid = input('갤러리 ID: ')
filename = '1' #= input('저장 파일명: ')
start = '0' #= input('시작 페이지(PC버전기준, 기본값=0): ')

if (start==''): i=0
else: i=int(start)


# 갤러리 링크
link = 'https://gall.dcinside.com/mgallery/board/lists/?id=tenbagger' #= 'https://gall.dcinside.com/board/lists/?id=' + gid

str_start = '2021-03-01 00:00:00.000000'
str_end = '2021-05-31 00:00:00.000000'
str_formatter = "%Y-%m-%d %H:%M:%S.%f"
date_start = datetime.datetime.strptime(str_start,str_formatter);
date_end = datetime.datetime.strptime(str_end,str_formatter);


print(str_start + '~' +str_end +'간의 데이터 수집.')
        
taskdone = False
trial = 0

prev_num = 0

while not taskdone and trial < 10:
    data = 'Post ID\tTitle\tNickname(IP)\tDate\tViewcount\tUpvoteCount\n'

    pidlst = []
    titlelst = []

    fin = False
    r = None

    while not fin:
        time.sleep(0.5)
        
        i += 1
        print('===== 페이지 읽는 중... [{}번째...] ====='.format(i))#, end='\r')
        titleok = False

        while not titleok:
            r = requests.get(link + '&page=' + str(i), headers = headers).text
            bs = BeautifulSoup(r, 'lxml')

            posts = bs.find_all('tr', class_='ub-content us-post')

            for p in posts:
                title = p.find('td', class_='gall_tit ub-word')

                # 공지 제외 (볼드태그 찾을때 str 처리 해줘야 찾기가능)
                if not '<b>' in str(title):
                    titleok = True
                    pid = p.find("td", {"class", "gall_num"}).text.strip()
                    title = midReturn(str(title), '</em>', '</a>')
                    nick = p.find("td", {"class", "gall_writer ub-writer"}).text.strip()
                    date = datetime.strptime(p.find('td', class_='gall_date').get('title'), "%Y-%m-%d %H:%M:%S")
                    view = p.find("td", {"class", "gall_count"}).text.strip()
                    recom = p.find("td", {"class", "gall_recommend"}).text.strip()
                    #print(pid + "\t" + str(date) + "\t" + re.repr(nick) + "\t\t" + re.repr(title))

                    # 이전 넘버링보다 현재 넘버링이 적을 경우에만 읽기
                    if prev_num > int(pid) or prev_num == 0:
                        
                        #초 단위까지는 안 가도록 함
                        if date >= ystday:
                            data += pid + "\t" + title + "\t" + nick + "\t" + str(date) + "\t" + view + "\t" + recom + "\n"
                            pidlst.append(pid)
                            titlelst.append(title)
                            

                        else:
                            fin = True
                            date = ystday
                            break
                            '''
                            ask = input('중지하시겠습니까? [Y/n]: ')
                            if ask == 'y' or ask == 'Y':
                                print('[NO]' + title)
                                print('기간 초과:', date)
                                fin = True
                                date = ystday
                                break
                            else:
                                data += pid + "\t" + title + "\t" + nick + "\t" + str(date) + "\t" + view + "\t" + recom + "\n"
                            '''

                        prev_num = int(pid)
                        
                    else:
                        print(pid + '\t올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.')
                    
            if not titleok:
                print('게시글 크롤링 실패. 5초 후 다시 시도해 봅니다.')
                #i -= 1
                time.sleep(5)
    print('저장 완료')
    taskdone = True


print("파일 쓰는 중...")
open(filename + '.tsv', 'w', encoding='utf-8-sig').write(data)
print("쓰기 완료.")

from google.colab import files
files.download(filename + '.tsv') 

#print(len(pidlst))
#print(len(titlelst))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
3285023	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285022	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285021	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285020	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285019	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285018	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285017	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285016	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285015	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285013	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285011	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285010	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285009	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285008	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285007	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285006	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285004	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285003	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285002	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285001	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3285000	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3284999	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3284998	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3284997	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3284996	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
3284995	올바르지 않은 글 넘버 - 무시하고 계속 읽습니다.
32

ConnectionError: ignored

In [2]:
from google.colab import files
files.download(filename + '.tsv') 

NameError: ignored

In [1]:

from google.colab import files
files.download('1.tsv') 

FileNotFoundError: ignored

In [ ]:
#from google.colab import files
#files.download(filename + '.tsv') 

from google.colab import files
files.download(filename + '.tsv') 

fulltitle ='';
for i in titlelst :
  fulltitle += (' ' + i)

p =regexz.findall('[a-zA-Z]+',fulltitle)

count={}
for i in p:
    try: count[i.upper()] += 1
    except: count[i.upper()]=1
#print(count)

sdict= sorted(count.items(), key=operator.itemgetter(1))

sdict.reverse()

##티커가 아닌것들으 제거 해야됨.
##

#print(type(sdict))

dellst = ['ETF','LT','GT','CEO','IPO','G','K','TXT','GIF','JPG','FDA','VS','MACD','RSI']
i=0
while i < len(sdict):
  if (sdict[i][0] in dellst ) :
     del sdict[i]
     i -=1
  i += 1

sdict30 = dict(sdict[0:10]) #해스피30
sdict30


{}

In [ ]:
hspiTicker = list(sdict30.keys())
#print(hspiTicker)
hspiRatio = []
sum=0
for i in hspiTicker :
  sum += sdict30[i]

for i in hspiTicker :
  sdict30[i] = sdict30[i] / sum

assets = web.get_data_yahoo(hspiTicker, start='1989-12-01', end='2021-06-02',interval='m')['Adj Close'].pct_change()

assets.index = assets.index.strftime("%Y-%m") 
assets.index = pd.to_datetime(assets.index).to_period("M")


#assets=ind10_ret.join(assets,how='left', lsuffix='', rsuffix='')
#assets.dropna(inplace=True)
assets = assets.fillna(0)

assets_mean = assets.mean()
assets_var = assets.var()
assets_cov = assets.cov() #assets_var

assets
sdict30

RemoteDataError: ignored

In [ ]:
### asset allocation weights

### 1.hspiRatio
hspiRatio


### 2.equal weight (EWI)
EWI = [1/30]*30

weight_df = pd.DataFrame([],columns = assets.columns)
weight_df.index.name = "Date"


### 백테스트
def _date_str_to_int(date):
    y, m = date.split('-')
    y, m = int(y), int(m)
    return y,m

def _cal_date_with_month(date, mon, oper='-'):
    y, m = _date_str_to_int(date)
    if oper == '-':
        if m-mon < 1:
            m= m-mon
            while m < 1:
              y -= 1
              m = m + 12
        else:
            m -= mon

    elif oper == '+':
        if m+mon > 12:
            m=m+mon
            while m > 12:
              y += 1
              m = m - 12
        else:
            m += mon

    return '{0:04d}-{1:02d}'.format(y,m)

def _df_to_estimate(self, df):
    """
        === Description ===

    함수 설명: 최적화 문제를 풀기 위해 필요한 estimate를 계산

    * df: estimate 계산에 이용

    output: estimate list 
            ex) return, variance, covariance matrix (추후에 list에 다 넣기에는 형식이 애매하면 dictionary를 이용하거나,
                                                    새로 class 작성해서 넘겨주는 방식도 고려해볼 수 있을수도)

    # 현재(11/19) backtest에서는 아래 형식으로 구성되었다고 가정하였습니다.
      est_lst[0]: mean lst ex) [0.5,0.3, ..., 0.7] 
      est_lst[1]: var lst ex) [...]
      est_lst[2]: cov matrix ex) [[...],[...],...]
      est_lst[3]: holding-period return 


    """
    n=4
    est_lst = [0 for i in range(n)]

    est_lst[0] = df.mean()
    est_lst[1] = df.var()
    est_lst[2] = df.cov()
    #lookback 기간동안 발생한 holding-period return
    s = {}
    for i in df.columns:
        val = df.get(i)
        hldprd_ret = 1.0
        for j in val : 
            j += 1   #ror -> total return 곱연산으로 기간의 수익을 가져오기 위함.
            hldprd_ret*=j
            s.update({i:hldprd_ret})

    est_lst[3] = pd.Series(s)
    est_lst[3] = est_lst[3] - 1 #total return -> ror
    
    return est_lst

def _pfo_return(mean, weight_lst):
    """
    === Description ===
    포트폴리오 return 계산

    * parameter *
    #est_mean: 특정 기간동안 estimate한 자산들의 평균 수익률 /type:Series
    mean: 수익률 /type:Series
    weight_lst: (가정)최적화를 통해 구한 weight / type:Series

    output: pfo return / type: float

    """
    ptf_ret = mean @ weight_lst

    return ptf_ret


inv_start = '2021-02'
inv_end = '2021-06'
amount = 1000
method = 'mention-weight'
lookback_period = 3

tmp_res = pd.DataFrame([], columns = ["{}_{}_return".format(hspiTicker, method),"{}_{}_balance".format(hspiTicker, method)])

size = len(assets.columns)
pfo_amount = np.array([1.0/size]*size)*amount

tmp_res.loc["Initial"] = [0, amount] #res_df 초기값

tmp_date = inv_start
tmp_end = _cal_date_with_month(inv_end, 1,'+')

while tmp_date != tmp_end:
  #look_start = _cal_date_with_month(tmp_date, lookback_period,'-')
  #lookback_df = assets.loc[look_start:tmp_date][:-1]
  #est_lst = self._df_to_estimate(lookback_df)

  j=0
  weight_dict = sdict30
  weight_lst = pd.Series(weight_dict)

  
  pfo_amount *= (assets.loc[tmp_date] + 1) #각 산업군 (amount * (1+수익률))
  print(tmp_date)
  print(weight_lst)
  print(assets.loc[tmp_date])
  s = _pfo_return(assets.loc[tmp_date], weight_lst)
  print(s)
  amount *= (s+1)
  print(amount)

  
  #print([_pfo_return(assets.loc[tmp_date], weight_lst), sum(pfo_amount)])
  #tmp_res.loc[tmp_date] = [_pfo_return(assets.loc[tmp_date], weight_lst), sum(pfo_amount)]   

  
  
  
  tmp_date = _cal_date_with_month(tmp_date, 1, '+')




### 퍼포먼스 이벨류에이션




